<div class="alert alert-info">Chicago Airbnb-Steiner bk2</div><p>
You and a group of friends are considering purchasing a property in Chicago that you can use as an investment. You have heard from other people that they have made a lot of money by renting out either a room or an entire unit (apartment or house). 
    <p>

Your friends ask you to analyze data so that they can understand how much you would charge per night based on the type of dwelling you were to purchase.

In [1]:
#bring in packages meeded
import pandas as pd
import numpy as np
import seaborn as sns

import joblib
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
#import the sql package
from sqlalchemy import create_engine

In [3]:
#set up variable for the db
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db for this class
user = r'postgres' # using the postgres user because I'm too lazy to create a new one
pw = r'BrightBlessings1!' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'cleaned' # schema 

In [4]:
#open the connection
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [5]:
#read from the cleaned bnb table 

table_name = r'bnb_cleaned'
schema = r'cleaned' # schema were the data was loaded last week.

#set df equal to bnb
bnb = pd.read_sql_table(table_name, db_conn, schema)
df=bnb

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6039 entries, 0 to 6038
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                6039 non-null   int64 
 1   room_type         6039 non-null   object
 2   price             6039 non-null   int64 
 3   minimum_nights    6039 non-null   int64 
 4   availability_365  6039 non-null   int64 
 5   minimum_cost      6039 non-null   int64 
 6   possible_income   6039 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 330.4+ KB


In [7]:
df.describe()

,id,price,minimum_nights,availability_365,minimum_cost,possible_income
count,6.039000e+03,6039.000000,6039.000000,6039.000000,6039.000000,6.039000e+03
mean,2.825647e+07,143.379533,6.759563,172.470608,950.981785,2.137809e+05
std,1.286957e+07,178.494246,11.839582,138.251715,2843.870952,6.518093e+05
min,2.384000e+03,10.000000,1.000000,0.000000,14.000000,0.000000e+00
25%,1.886402e+07,65.000000,1.000000,35.000000,105.000000,4.617500e+03
50%,3.018688e+07,100.000000,2.000000,159.000000,237.000000,2.717600e+04
75%,3.962737e+07,156.000000,3.000000,326.000000,623.500000,1.080960e+05
max,4.551558e+07,3690.000000,90.000000,365.000000,114390.000000,1.720992e+07


<div class="alert alert-info">3-Way splitting of the data</div>



In [8]:
#gather up names of all the columns
cols = df.columns

#set the prediction column and the feature columns for KNN
#using room type
prediction_col = 'room_type'
feature_cols = [c for c in cols if c != prediction_col]
X = df[feature_cols]
y = df[prediction_col]

In [9]:
# Train-Validation-Test Split - First split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, random_state=42, test_size=0.3, stratify=y)

checking the data as I go along

In [10]:
X_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1812 entries, 2819 to 2890
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                1812 non-null   int64
 1   price             1812 non-null   int64
 2   minimum_nights    1812 non-null   int64
 3   availability_365  1812 non-null   int64
 4   minimum_cost      1812 non-null   int64
 5   possible_income   1812 non-null   int64
dtypes: int64(6)
memory usage: 99.1 KB


In [11]:
X_temp.head()

,id,price,minimum_nights,availability_365,minimum_cost,possible_income
2819,28819602,180,2,4,360,1440
3407,33662870,70,2,269,140,37660
2571,27106400,96,1,59,96,5664
917,12698881,159,1,320,159,50880
4682,40232559,58,4,75,232,17400


In [12]:
y_temp.info()

<class 'pandas.core.series.Series'>
Index: 1812 entries, 2819 to 2890
Series name: room_type
Non-Null Count  Dtype 
--------------  ----- 
1812 non-null   object
dtypes: object(1)
memory usage: 28.3+ KB


In [13]:
y_temp.head()

2819    Entire home/apt
3407       Private room
2571    Entire home/apt
917     Entire home/apt
4682    Entire home/apt
Name: room_type, dtype: object

In [14]:
# Train-Validation-Test Split - Second split
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, random_state=42, test_size=0.5, stratify=y_temp)

In [15]:
print(X_test.shape)
X_test.head()

(906, 6)


,id,price,minimum_nights,availability_365,minimum_cost,possible_income
700,9150564,75,3,177,225,39825
1297,16488400,350,5,88,1750,154000
5572,43835556,101,2,167,202,33734
3674,35382254,635,1,102,635,64770
3676,35387889,119,3,179,357,63903


Yes, I finally figured out what I was actually supposed to do.  :)

In [16]:
joblib.dump(X_val, 'x_val.jolib')
print(X_val.shape)
X_val.head()

(906, 6)


,id,price,minimum_nights,availability_365,minimum_cost,possible_income
766,10411427,39,3,294,117,34398
430,6374350,76,2,345,152,52440
3610,34942855,208,2,13,416,5408
3217,32384583,75,2,358,150,53700
2950,29662221,35,32,295,1120,330400


In [17]:
print(y_test.shape)
y_test.head()

(906,)


700        Private room
1297    Entire home/apt
5572    Entire home/apt
3674    Entire home/apt
3676    Entire home/apt
Name: room_type, dtype: object

In [18]:
joblib.dump(y_val, 'y_val.joblib')
print(y_val.shape)
y_val.head()

(906,)


766        Private room
430        Private room
3610    Entire home/apt
3217    Entire home/apt
2950       Private room
Name: room_type, dtype: object

Okay, build the first model and save
### Building the Model
Following the 3 way split example from work book

In [19]:
# Train a Random Forest classifier using training set
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [20]:
pred_X_test = model.predict(X_test)

In [21]:
accuracy_score(y_test,pred_X_test)

0.8598233995584988

<div class="alert alert-info">Check different features to see which performs best.</div>

<br>It appears that the accuracy is fairluy good so far.  
<br>
I did try using different features features, however, the other features yielded errors.
<br>due to the errors, I chose to go with room type

In [22]:
model_filename = "bnb_rf_model.joblib"
joblib.dump(model, model_filename)

['bnb_rf_model.joblib']


<p> I am shamelessly using the code from week 6
